In [ ]:
from argparse import Namespace
import re

import numpy as np
import pandas
import time
import pandas as pd
import regex as re
import text_processing as tp
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

Pre-defined params

In [ ]:
test_proportion = 0.15
eval_proportion = 0.15

In [ ]:
batch_size = 256
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 7
hidden_size = 16
num_channels = 128
embedding_size = 32

lr = 0.01

In [ ]:
device

In [ ]:
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device='cpu'):
    dataloader = DataLoader(dataset, batch_size, shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for key, tensor in data_dict.items():
            out_data_dict[key] = data_dict[key].to(device) # Sending tensors to propper device
        yield out_data_dict

In [ ]:
class SentimentAnalyzerMLP(nn.Module):
    def __init__(self, input_cnt, output_cnt, hidden_size):
        super().__init__()
        self.flatten = nn.Flatten(start_dim=1)
        self.linear_stack = nn.Sequential(
            nn.Linear(input_cnt, hidden_size),
            nn.ELU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ELU(),
            nn.Linear(hidden_size, output_cnt)
        )
    def forward(self, x_data, apply_softmax=False):
        y_out = self.linear_stack(x_data)
        if apply_softmax:
            y_out = nn.functional.softmax(y_out, dim=1)
        return y_out

In [ ]:
class SentimentAnalyzerCNN(nn.Module):
    def __init__(self, in_channels, num_channels, num_classes):
        super().__init__()
        self.convnet = nn.Sequential(
            nn.Conv1d(in_channels, num_channels, kernel_size=3),
            nn.ELU(),
            nn.Conv1d(num_channels, num_channels, kernel_size=3, stride=2),
            nn.ELU(),
            nn.Conv1d(num_channels, num_channels, kernel_size=3, stride=2),
            nn.ELU(),
            nn.Conv1d(num_channels, num_channels, kernel_size=3),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(num_channels, num_classes)

    def forward(self, x_data, apply_softmax=False):
        fratures = self.convnet(x_data).squeeze(dim=2)
        y_out = self.fc(fratures)
        if apply_softmax:
            y_out = torch.nn.functional.softmax(y_out, dim=1)
        return y_out

In [ ]:
class SentimentAnalyzerEmbedCNN(nn.Module):
    def __init__(self, embedding_size, num_embeddings,\
                 num_channels, hidden_size, num_classes, kernel_size=3, pretrained_embeddings = None, padding_idx=0):
        super().__init__()
        
        if pretrained_embeddings is None:
            self.embed = nn.Embedding(num_embeddings, embedding_size, padding_idx)
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.embed = nn.Embedding(num_embeddings, embedding_size, padding_idx, _weight=pretrained_embeddings)

        self.convnet = nn.Sequential(
            nn.Conv1d(embedding_size, num_channels, kernel_size=kernel_size),
            nn.ELU(),
            nn.Conv1d(num_channels, num_channels, kernel_size=kernel_size, stride=2),
            nn.ELU(),
            nn.Conv1d(num_channels, num_channels, kernel_size=kernel_size, stride=2),
            nn.ELU(),
            nn.Conv1d(num_channels, num_channels, kernel_size=kernel_size),
            nn.ELU(),
            nn.AdaptiveAvgPool1d(1),
            )
        self.fc = nn.Sequential(
            nn.Linear(num_channels, hidden_size),
            nn.Linear(hidden_size, num_classes)
        )
        
    def forward(self, x_data, apply_softmax=False):
        # x_data - vector of indices
        embed_vectors = self.embed(x_data).permute(0, 2, 1) # permutation to make embedding_dimensionality input_channels in CNN.
                                                            # now each token embedding vector are collumn, not a row
        
        features = self.convnet(embed_vectors).squeeze(dim=2)
        y_out = self.fc(features)
        if apply_softmax:
            y_out = nn.functional.softmax(y_out, dim=1)

        return y_out

In [ ]:
df = pd.read_csv('C:/Files/Datasets/twitter_financial_news_sentiment/sent_train.csv')
# Максимум в отзывах 31 слово, разделенных пробелом. Max 190 символов

In [ ]:
df = df.rename(columns={'text' : 'x_data', 'label' : 'y_target'})
df['y_target'] = df['y_target'].apply(str)

setting train, test, evaluation split

In [ ]:
df['split'] = 'train'
df_len = len(df)
test_eval_idx = np.random.choice(df_len, int(df_len*(test_proportion + eval_proportion)), replace=False)

test_eval_prop = test_proportion / eval_proportion
val_len = int(len(test_eval_idx)/(test_eval_prop+1))

In [ ]:
for i in range(test_eval_idx.size):
    if i < val_len:
        df.loc[test_eval_idx[i], 'split'] = 'validation'
    else:
        df.loc[test_eval_idx[i], 'split'] = 'test'

Замена адреса

In [ ]:
df['x_data'] = df['x_data'].apply(lambda x: re.sub(r'https?://.*', r'SOMEURL', x))

In [ ]:
tokenizer = tp.SeparatorTokenizer()

Первое заполнение словаря и сохранение в файл

In [ ]:
tokens_vocabulary = tp.Vocabulary()
label_vocabulary = tp.Vocabulary(is_lexical_tokens=False)

max_sentence_len = 0

for i in range(len(df)):
    cur_len = len(tokens_vocabulary.add_tokens(tokenizer.tokenize(df.loc[i, 'x_data']))) # returns list of new token indices
    max_sentence_len = max(max_sentence_len, cur_len)
    label_vocabulary.add_token(str(df.loc[i, 'y_target']))

tokens_vocabulary.to_json('tokens_vocab.json')
label_vocabulary.to_json('label_vocab.json')

In [ ]:
tokens_vocabulary = tp.Vocabulary().from_json('tokens_vocab.json')
label_vocabulary = tp.Vocabulary().from_json('label_vocab.json')
vectorizer = tp.Vectorizer(tokens_vocabulary, label_vocabulary, max_sentence_len) # Необходимо знать max_sentence_len для использования сверточной нн

In [ ]:
dataset = tp.CustomDataset(df, tokenizer, vectorizer)
batch_generator = generate_batches(dataset, batch_size, device=device)

In [ ]:
# model = SentimentAnalyzerMLP(len(tokens_vocabulary), len(label_vocabulary), hidden_size)
# model = SentimentAnalyzerCNN(tokens_vocabulary.size(), const_num_channels, label_vocabulary.size())
model = SentimentAnalyzerEmbedCNN(embedding_size=embedding_size, num_embeddings=len(tokens_vocabulary._token_to_idx),\
                                  num_channels=num_channels, hidden_size=hidden_size, num_classes=3, kernel_size=3)
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def compute_accuracy(prediction, target):
    prediction_lables = prediction.max(dim=1)[1]
    target_labels = target.max(dim=1)[1]
    n_correct = torch.eq(prediction_lables, target_labels).sum().item()
    return (100*n_correct)/len(prediction_lables)

In [ ]:
for epoch in range(epochs):
    epoch_err = train_running_acc = validation_acc = 0
    batch_generator = generate_batches(dataset, batch_size, device=device)
    model.train()
    dataset.set_dataframe_split('train')
    for idx, batch in enumerate(batch_generator):
        optimizer.zero_grad()

        prediction = model(batch['x_data'])

        print(batch['y_target'].size())
        loss = loss_fn(prediction, batch['y_target'])

        epoch_err += loss.item()
        train_running_acc += (compute_accuracy(prediction, batch['y_target'])-train_running_acc)/(idx+1)

        loss.backward()

        optimizer.step()

    # evaluating model perfomance each epoch
    model.eval()
    dataset.set_dataframe_split('validation')
    batch_generator = generate_batches(dataset, batch_size, device=device)
    for idx, batch in enumerate(batch_generator):
        prediction = model(batch['x_data'])
        validation_acc += (compute_accuracy(prediction, batch['y_target'])-validation_acc)/(idx+1)

    print('epoch: ', epoch+1)
    print('train accuracy: ', train_running_acc)
    print('validation accuracy: ', validation_acc)
    print('-'*30)
    

256 = 73.57954545454545\
128 = 76.953125